<span style="font-weight:bold; font-size: 3rem; color:#0047AB;">Feature backfill of weather and electricity data</span>

## 🗒️ This notebook is divided into the following sections:

1. Getting feature data from weather and electricity data APIs
2. Reformat and process raw data
3. Create data expectations to ensure robust data
4. Upload processed and robust data to feature groups on Hopsworks

## <span style='color:#ff5f27'> Imports</span>

In [54]:
import pandas as pd
import os
import numpy as np
import requests
import datetime
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import datetime
import time
import json
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
from matplotlib.ticker import MultipleLocator
import openmeteo_requests
import requests_cache
from retry_requests import retry
import hsfs
from pathlib import Path
from dotenv import load_dotenv
import hopsworks
import sys

root_dir = Path().resolve().parent
sys.path.append(str(root_dir))

from format_data import format_weather_data, format_price_data, process_weather_data
from get_electricity_prices import get_data
from get_weather_data import get_historical_weather, get_weather_forecast
from entsoe_data import fetch_historical_data, ensure_valid_series



In [55]:
import hsfs

In [56]:
import hopsworks

### connect to Hopsworks and Entose

In [57]:
load_dotenv()
entose_api = os.getenv("ENTSOE_API")
hopsworks_api = os.getenv("HOPSWORKS_API_KEY")

os.environ["HOPSWORKS_API_KEY"] = hopsworks_api

project = hopsworks.login()
print(f"Connected to project: {project.name}")

2025-01-07 11:03:18,628 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-01-07 11:03:18,719 INFO: Initializing external client
2025-01-07 11:03:18,720 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-01-07 11:03:20,190 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1169556
Connected to project: sthaji


## <span style='color:#ff5f27'> STEP 1: Get weather and electricity (entsoe) data</span>

### Weather data

In [ ]:
start_date = "2022-11-01"
end_date = "2025-01-07"

end_date = shifted_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime("%Y-%m-%d")
print(f"Start date: {start_date}, End date: {end_date}")
hist_weather_df = get_historical_weather("Stockholm", "2022-11-01", "2025-01-06", 59.3294, 18.0687)
formatted_hist_weather_df = process_weather_data(hist_weather_df)

Start date: 2022-11-01, End date: 2025-01-08
2022-11-01
2025-01-06
Coordinates 59.29701232910156°N 18.163265228271484°E
Elevation 24.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
MAX DATE
2024-12-31


In [59]:
#print(formatted_hist_weather_df.head())
#print(formatted_hist_weather_df.dtypes)
print(formatted_hist_weather_df['date'].tail(1))

print(formatted_hist_weather_df['date'].max())

19007   2024-12-31 23:00:00
Name: date, dtype: datetime64[ns]
2024-12-31 23:00:00


### Entsoe data

In [60]:
entsoe_raw = fetch_historical_data(entose_api, start_date, end_date)

[DEBUG] Successfully fetched load data for finland.
[DEBUG] Successfully fetched load data for norway.
[DEBUG] Successfully fetched load data for denmark.
[DEBUG] Successfully fetched cross-border flows SE3 to finland.
[DEBUG] Successfully fetched cross-border flows finland to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to norway.
[DEBUG] Successfully fetched cross-border flows norway to SE3.
[DEBUG] Successfully fetched cross-border flows SE3 to denmark.
[DEBUG] Successfully fetched cross-border flows denmark to SE3.


In [61]:
#print(formatted_hist_weather_df.head())
#print(entsoe_df.head())
entsoe_df = entsoe_raw.copy()
entsoe_df.columns = entsoe_df.columns.str.lower()
entsoe_df.columns = entsoe_df.columns.str.lower().str.replace(' ', '_').str.replace('-', '_')
entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])

## <span style='color:#ff5f27'> STEP 2: Reformat data and feature engineering</span>

### Code to reformat/drop columns for Entsoe

In [ ]:
def modify_entsoe_df(df):
    columns_to_merge = [
        'flows_se3_to_finland', 'flows_finland_to_se3',
        'flows_se3_to_norway', 'flows_norway_to_se3',
        'flows_se3_to_denmark', 'flows_denmark_to_se3'
    ]

    df['imported_energy'] = df.apply(
        lambda row: sum(-row[col] if 'se3_to' in col else row[col] for col in columns_to_merge),
        axis=1
    )

    df = df.drop(columns=columns_to_merge)

    df['total_load'] = ( df['load_finland'] +
        df['load_norway'] + df['load_denmark'] + df['load_se3']
    )

    df = df.drop(columns=['load_finland', 'load_norway', 'load_denmark', 'load_se3'])

    columns_to_sum = [
        'total_generation_biomass', 'total_generation_fossil_gas',
        'total_generation_fossil_hard_coal', 'total_generation_fossil_oil',
        'total_generation_hydro_run_of_river_and_poundage',
        'total_generation_other_renewable', 'total_generation_solar',
        'total_generation_waste', 'total_generation_wind_offshore',
        'total_generation_wind_onshore'
    ]

    df['total_generation_se'] = df[columns_to_sum].sum(axis=1)
    df = df.drop(columns=columns_to_sum)

    return df

### Code to reformat weather data

In [63]:
def modify_weather_df(df):
    df.drop(['snow_depth',
       'pressure_msl', 'cloud_cover', 'wind_speed_10m',
       'wind_direction_10m', 'wind_direction_100m', 'city'], axis=1, inplace=True)
    return df

### Execute formattings

In [64]:
entsoe_df = modify_entsoe_df(entsoe_df)
formatted_hist_weather_df = modify_weather_df(formatted_hist_weather_df)

In [14]:
print(entsoe_df.dtypes)

prices                        float64
date                   datetime64[ns]
imported_energy               float64
total_load                    float64
total_generation_se           float64
dtype: object


### Create lagging columns

Creating lagging columns to capture trends better. This code only adds lagging values for those data rows that actually have a previous data point of 1 hour before, ensuring more robust and reliable data.

In [ ]:


def create_lagging_columns(df):

    df['date'] = pd.to_datetime(df['date'])
    df['date_diff'] = df['date'].diff().dt.total_seconds() / 3600

    original_columns = df.columns.tolist()

    for column in original_columns:
        if column != 'date' and column != 'date_diff':
            df[column + '_lag'] = df[column].shift(1).where(df['date_diff'] == 1)

    # print(df.head())

    df = df.drop(columns=['date_diff'])
    print("number of rows before dropping na: ", df.shape[0])
    df = df.dropna()
    print("number of rows after dropping na: ", df.shape[0])
    # print(df.head())
    return df

### Function that shifts price data back once

Explanation: Since the aim of our model is to predict the next hours' electricity price, we must ensure the training is made on this as well. Therefore a new column "future price" is created to make sure the model is trained appropriately for its task. 

In [ ]:
def add_future_price_column(df, price_column='prices'):
    if price_column not in df.columns:
        raise ValueError(f"Column '{price_column}' does not exist in the DataFrame")

    df['future_price'] = df[price_column].shift(-1)
    df = df.dropna()

    return df

### excecuting modifications from above functions

In [ ]:


entsoe_df = create_lagging_columns(entsoe_df)
formatted_hist_weather_df = create_lagging_columns(formatted_hist_weather_df)
add_future_price_column(entsoe_df)

for column in entsoe_df.columns:
    if column != 'date':
        entsoe_df[column] = pd.to_numeric(entsoe_df[column], errors='coerce').astype('float')



number of rows before dropping na:  19143
number of rows after dropping na:  19137
number of rows before dropping na:  18504
number of rows after dropping na:  18501


In [19]:
print(entsoe_df.dtypes)
entsoe_df = entsoe_df.dropna()
entsoe_df

prices                            float64
date                       datetime64[ns]
imported_energy                   float64
total_load                        float64
total_generation_se               float64
prices_lag                        float64
imported_energy_lag               float64
total_load_lag                    float64
total_generation_se_lag           float64
future_price                      float64
dtype: object


,prices,date,imported_energy,total_load,total_generation_se,prices_lag,imported_energy_lag,total_load_lag,total_generation_se_lag,future_price
index,,,,,,,,,,
2022-11-01 00:00:00+00:00,76.68,2022-11-01 01:00:00,-2149.0,20662.00,2497.0,80.65,-2167.0,21187.00,2088.0,55.26
2022-11-01 01:00:00+00:00,55.26,2022-11-01 02:00:00,-2133.0,20351.00,2917.0,76.68,-2149.0,20662.00,2497.0,37.00
2022-11-01 02:00:00+00:00,37.00,2022-11-01 03:00:00,-2125.0,20363.00,3313.0,55.26,-2133.0,20351.00,2917.0,38.76
2022-11-01 03:00:00+00:00,38.76,2022-11-01 04:00:00,-2131.0,20661.00,3575.0,37.00,-2125.0,20363.00,3313.0,54.63
2022-11-01 04:00:00+00:00,54.63,2022-11-01 05:00:00,-1855.0,22036.00,3638.0,38.76,-2131.0,20661.00,3575.0,59.95
...,...,...,...,...,...,...,...,...,...,...
2025-01-06 13:00:00+00:00,25.61,2025-01-06 14:00:00,-2096.0,35074.75,4700.0,28.60,-1980.0,35077.00,4526.0,26.32
2025-01-06 14:00:00+00:00,26.32,2025-01-06 15:00:00,-1879.0,35484.75,4858.0,25.61,-2096.0,35074.75,4700.0,26.87
2025-01-06 15:00:00+00:00,26.87,2025-01-06 16:00:00,-1707.0,35887.50,4832.0,26.32,-1879.0,35484.75,4858.0,31.66


## <span style='color:#ff5f27'> STEP 3: Create expectations before uploading to feature group</span>

### Weather data expectations

In [20]:
import great_expectations as ge

weather_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="weather_expectation_suite"
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"temperature_2m",
            "min_value":-100.0,
            "max_value":500.0,
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"temperature_2m_lag",
            "min_value":-100.0,
            "max_value":500.0,
        }
    )
)


weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"precipitation",
            "min_value":-5.0,
            "max_value":500.0,

        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"precipitation_lag",
            "min_value":-5.0,
            "max_value":500.0,

        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"wind_speed_100m",
            "min_value":-0.1,
            "max_value":1000,
            "strict_min":True
        }
    )
)

weather_expectation_suite.add_expectation(
    ge.core.ExpectationConfiguration(
        expectation_type="expect_column_min_to_be_between",
        kwargs={
            "column":"wind_speed_100m_lag",
            "min_value":-0.1,
            "max_value":1000,
            "strict_min":True
        }
    )
)

{"expectation_type": "expect_column_min_to_be_between", "kwargs": {"column": "wind_speed_100m_lag", "min_value": -0.1, "max_value": 1000, "strict_min": true}, "meta": {}}

### Entsoe data expectations

In [ ]:

import great_expectations as ge

data_expectation_suite = ge.core.ExpectationSuite(
    expectation_suite_name="data_expectation_suite"
)
columns = [
    'prices', 'imported_energy', 'total_load','total_generation_se',
    'prices_lag', 'imported_energy_lag', 'total_load_lag','total_generation_se_lag',
    'date','future_price'
] 

min_max_values = {
    'total_load': (-0.1, 60000),
    'total_load_lag': (-0.1, 60000),
    'imported_energy': (-30000, 30000),
    'imported_energy_lag': (-30000, 30000),
    'total_generation_se': (-0.1, 100000),
    'total_generation_se_lag': (-0.1, 100000),
    'prices': (-1000, 1000),
    'prices_lag': (-1000, 1000),
    'future_price': (-1000, 1000),
}

for column, (min_value, max_value) in min_max_values.items():
    data_expectation_suite.add_expectation(
        ge.core.ExpectationConfiguration(
            expectation_type="expect_column_min_to_be_between",
            kwargs={
                "column": column,
                "min_value": min_value,
                "max_value": max_value,
                "strict_min": True
            }
        )
    )

    print(data_expectation_suite)

{
  "expectation_suite_name": "data_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "total_load",
        "min_value": -0.1,
        "max_value": 60000,
        "strict_min": true
      },
      "meta": {}
    }
  ],
  "data_asset_type": null,
  "meta": {
    "great_expectations_version": "0.18.12"
  }
}
{
  "expectation_suite_name": "data_expectation_suite",
  "ge_cloud_id": null,
  "expectations": [
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "total_load",
        "min_value": -0.1,
        "max_value": 60000,
        "strict_min": true
      },
      "meta": {}
    },
    {
      "expectation_type": "expect_column_min_to_be_between",
      "kwargs": {
        "column": "total_load_lag",
        "min_value": -0.1,
        "max_value": 60000,
        "strict_min": true
      },
      "meta": {}
    }
  ],
  "da

## <span style='color:#ff5f27'> STEP 4: Upload data frames to feature groups</span>

In [23]:
try:
    fs = project.get_feature_store()
    print(f"Connected to feature store: {fs.name}")
except Exception as e:
    print(f"Failed to get feature store: {e}")

Connected to feature store: oskaralf_featurestore


### Creating feature groups

In [24]:
weather_fg = fs.get_or_create_feature_group(
    name='weather_data_3',
    description='Weather data for SE3',
    version=1,
    primary_key=['date'],
    event_time='date', 
    expectation_suite=weather_expectation_suite
)

In [25]:
entsoe_fg = fs.get_or_create_feature_group(
    name='entsoe_data_3',
    description='Entsoe data for SE3',
    version=1,
    primary_key=['date'],
    event_time='date',
    expectation_suite=data_expectation_suite
)

### Inserting feature groups

In [26]:
formatted_hist_weather_df = formatted_hist_weather_df.dropna()
weather_fg.insert(formatted_hist_weather_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393654
2025-01-06 21:48:34,726 INFO: 	6 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393654


Uploading Dataframe: 100.00% |██████████| Rows 18501/18501 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/weather_data_3_1_offline_fg_materialization/executions


(Job('weather_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "temperature_2m_lag",
           "min_value": -100.0,
           "max_value": 500.0
         },
         "meta": {
           "expectationId": 696123
         }
       },
       "result": {
         "observed_value": -17.53499984741211,
         "element_count": 18501,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:48:34.000725Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min

In [ ]:
entsoe_df = entsoe_df.dropna()

entsoe_df['date'] = pd.to_datetime(entsoe_df['date'])
entsoe_fg.insert(entsoe_df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393655
2025-01-06 21:48:52,754 INFO: 	9 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1164446/fs/1155149/fg/1393655


Uploading Dataframe: 100.00% |██████████| Rows 19136/19136 | Elapsed Time: 00:02 | Remaining Time: 00:00


Launching job: entsoe_data_3_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1164446/jobs/named/entsoe_data_3_1_offline_fg_materialization/executions


(Job('entsoe_data_3_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "prices_lag",
           "min_value": -1000,
           "max_value": 1000,
           "strict_min": true
         },
         "meta": {
           "expectationId": 696127
         }
       },
       "result": {
         "observed_value": -440.1,
         "element_count": 19136,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-01-06T08:48:52.000753Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_co